In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import gkey

In [2]:
weather_data = pd.read_csv("../output_data/city_weather.csv")
weather_data

,City,Country,Cloudiness,Dates,Max Temperature,Latitude,Longitude,Humidity,Wind Speed
0,Rikitea,PF,100,1603058953,74.86,-23.12,-134.97,80,17.58
1,Souillac,MU,20,1603059027,73.40,-20.52,57.52,73,10.29
2,Nuuk,GL,45,1603059028,44.60,64.18,-51.72,75,25.28
3,Longyearbyen,SJ,20,1603059029,26.60,78.22,15.64,73,9.17
4,Saldanha,ZA,100,1603059031,57.20,-33.01,17.94,76,4.70
...,...,...,...,...,...,...,...,...,...
550,Panubigan,PH,92,1603059924,78.15,7.43,123.35,84,3.94
551,Uvarovka,RU,87,1603059925,36.19,55.53,35.61,90,12.19
552,Augusto Corrêa,BR,96,1603059926,78.44,-1.02,-46.63,78,5.21
553,Zhangye,CN,82,1603059927,41.74,38.93,100.45,53,4.76


### Humidity Heatmap
* Configure gmaps
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map

In [18]:
# configure gmaps
gmaps.configure(api_key=gkey)

# Store lat and lng as well as humidity
locations = weather_data[['Latitude', "Longitude"]]
humidity = weather_data["Humidity"]

In [19]:
# Heatmap Plot
heat_fig = gmaps.figure(center=(45, -10), zoom_level=2)
max_humidity = np.max(humidity)

# Heat layer added
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=6)

# add layer
heat_fig.add_layer(heat_layer)
heat_fig


Figure(layout=FigureLayout(height='420px'))

In [20]:
# Find Perfect Weather - Max Temp between 70-80, wind speed less than 10 mph, no clouds
weather_criteria_df = weather_data.loc[(weather_data["Max Temperature"] >=70) & \
                                       (weather_data["Max Temperature"] <=80) & \
                                        (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()
weather_criteria_df

,City,Country,Cloudiness,Dates,Max Temperature,Latitude,Longitude,Humidity,Wind Speed
30,Abu Kamal,SY,0,1603059069,70.63,34.45,40.92,28,7.52
34,Salalah,OM,0,1603059075,75.20,17.02,54.09,38,4.70
53,Sharjah city,AE,0,1603059102,78.80,25.36,55.40,21,4.70
157,Sesheke,ZM,0,1603059239,73.40,-17.48,24.30,20,4.70
175,Dubai,AE,0,1603059143,78.80,25.26,55.30,16,3.36
192,Hīt,IQ,0,1603059285,70.61,33.64,42.83,31,5.46
195,Noumea,NC,0,1603059204,77.00,-22.28,166.46,38,6.93
262,Voh,NC,0,1603059381,76.41,-20.97,164.70,58,4.68
286,Tura,IN,0,1603059411,72.34,25.52,90.22,91,1.79
289,Lumeje,AO,0,1603059415,72.34,-11.55,20.78,51,2.98


In [21]:
# Hotel dataframe
# Create a hotel_df
hotel_df = weather_criteria_df.loc[:,["City","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
30,Abu Kamal,SY,34.45,40.92,
34,Salalah,OM,17.02,54.09,
53,Sharjah city,AE,25.36,55.40,
157,Sesheke,ZM,-17.48,24.30,
175,Dubai,AE,25.26,55.30,
192,Hīt,IQ,33.64,42.83,
195,Noumea,NC,-22.28,166.46,
262,Voh,NC,-20.97,164.70,
286,Tura,IN,25.52,90.22,
289,Lumeje,AO,-11.55,20.78,


In [22]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": gkey}

In [23]:
print("Retrieving results for closest hotel in cities with the best weather.")
print("-------------------------")
for index, row in hotel_df.iterrows():
    # get info from dataframe
    city_name = row["City"]
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # adding keyword for params
    params["location"] = f'{lat}, {lng}'
    
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-------------------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)
print("-----End of Search-----")

Retrieving results for closest hotel in cities with the best weather.
-------------------------
Closest hotel in Abu Kamal is مطعم السلام.
-------------------------
Closest hotel in Salalah is Alpha lodge.
-------------------------
Closest hotel in Sharjah city is TIME Express Hotel.
-------------------------
Closest hotel in Sesheke is Protea Hotel by Marriott Zambezi River Lodge.
-------------------------
Closest hotel in Dubai is Queen Elizabeth 2.
-------------------------
Missing field/result... skipping.
-------------------------
Closest hotel in Noumea is GONDWANA HOTEL.
-------------------------
Closest hotel in Voh is Le gîte du Koniambo.
-------------------------
Closest hotel in Tura is Hotel Polo Orchid.
-------------------------
Missing field/result... skipping.
-------------------------
Closest hotel in Assiut is Assiut hotels Armed Forces.
-------------------------
Closest hotel in Santa Maria is Itaimbé Palace Hotel.
-------------------------
Missing field/result... ski

In [25]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
30,Abu Kamal,SY,34.45,40.92,مطعم السلام
34,Salalah,OM,17.02,54.09,Alpha lodge
53,Sharjah city,AE,25.36,55.40,TIME Express Hotel
157,Sesheke,ZM,-17.48,24.30,Protea Hotel by Marriott Zambezi River Lodge
175,Dubai,AE,25.26,55.30,Queen Elizabeth 2
192,Hīt,IQ,33.64,42.83,
195,Noumea,NC,-22.28,166.46,GONDWANA HOTEL
262,Voh,NC,-20.97,164.70,Le gîte du Koniambo
286,Tura,IN,25.52,90.22,Hotel Polo Orchid
289,Lumeje,AO,-11.55,20.78,


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat_fig.add_layer(marker_layer)

# Display figure
heat_fig

Figure(layout=FigureLayout(height='420px'))